In [6]:
import sys, pandas as pd, numpy as np
sys.path.append('..')
from src.task2_config import DATA_PATH
from src.loader import load_real_data
from src.chunker import chunk_all_complaints
from src.embedder import generate_all_embeddings
from src.vectorstore import create_vectorstore, search
from src.task2_reporter import create_final_summary
print("✅ Setup complete - Processing ALL your real data")

✅ Setup complete - Processing ALL your real data


In [11]:
# Cell: Debug - Check your processed file
import pandas as pd

df_check = pd.read_csv('../data/processed/filtered_complaints.csv')
print(f"📊 Shape: {df_check.shape}")
print(f"\n📋 First few rows:")
print(df_check.head(3))
print(f"\n📊 Product distribution:")
print(df_check['Product_Category'].value_counts())

📊 Shape: (1814, 8)

📋 First few rows:
   Complaint ID        Date received  \
0      12351447  1741219200000000000   
1      12203565  1740528000000000000   
2      14089944  1749945600000000000   

                                             Product  \
0  Credit reporting or other personal consumer re...   
1  Credit reporting or other personal consumer re...   
2                                           Mortgage   

                                               Issue  \
0      Problem with fraud alerts or security freezes   
1               Incorrect information on your report   
2  Applying for a mortgage or refinancing an exis...   

                                  Company State  \
0  TRANSUNION INTERMEDIATE HOLDINGS, INC.    TX   
1     Experian Information Solutions Inc.    FL   
2          Lennar Financial Services, LLC    IL   

                        Consumer complaint narrative  Text_Length  
0  XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX Apt XX...          317  
1  Subject

KeyError: 'Product_Category'

In [10]:
# Cell 2: Load YOUR processed data from Task 1
from pathlib import Path

processed_file = Path('../data/processed/filtered_complaints.csv')
print(f"📂 Loading from: {processed_file}")
df = pd.read_csv(processed_file)
print(f"✅ Loaded {len(df):,} processed complaints")
print(f"📊 Products:\n{df['Product_Category'].value_counts()}")

📂 Loading from: ..\data\processed\filtered_complaints.csv
✅ Loaded 1,814 processed complaints


KeyError: 'Product_Category'

In [2]:
df = load_real_data()
print(f"\n📊 Processing ALL {len(df):,} complaints")
print(f"📋 Columns: {df.columns.tolist()}")

📂 Loading YOUR real data: d:\10 acadamy\Intelligent Complaint Analysis for Financial Services\notebooks\..\data\processed\filtered_complaints.csv
✅ Loaded 1,814 complaints
📋 Columns: ['Complaint ID', 'Date received', 'Product', 'Issue', 'Company', 'State', 'Consumer complaint narrative', 'Text_Length']

📊 Processing ALL 1,814 complaints
📋 Columns: ['Complaint ID', 'Date received', 'Product', 'Issue', 'Company', 'State', 'Consumer complaint narrative', 'Text_Length']


In [4]:
import sys, pandas as pd
sys.path.append('..')
from src.task2_config import DATA_PATH

# Load and inspect
df = pd.read_csv(DATA_PATH)
print(f"✅ Loaded {len(df):,} complaints")
print(f"\n📋 COLUMNS:")
for col in df.columns:
    print(f"  • '{col}'")
    
# Check the narrative column
narrative_col = 'Consumer complaint narrative'
if narrative_col in df.columns:
    print(f"\n📝 Sample narrative (first 100 chars):")
    print(df[narrative_col].iloc[0][:100] if pd.notna(df[narrative_col].iloc[0]) else "EMPTY")
    
    # Count non-empty narratives
    non_empty = df[narrative_col].notna().sum()
    print(f"\n📊 Complaints with narratives: {non_empty:,}/{len(df):,} ({(non_empty/len(df)*100):.1f}%)")

✅ Loaded 1,814 complaints

📋 COLUMNS:
  • 'Complaint ID'
  • 'Date received'
  • 'Product'
  • 'Issue'
  • 'Company'
  • 'State'
  • 'Consumer complaint narrative'
  • 'Text_Length'

📝 Sample narrative (first 100 chars):
XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX Apt XXXX, XXXX, TX XXXX XXXX : XX/XX/XXXX TransUnion Consume

📊 Complaints with narratives: 1,814/1,814 (100.0%)


In [3]:
chunks_df = chunk_all_complaints(df)
print(f"\n📊 Created {len(chunks_df):,} total chunks")
print(f"📈 Avg chunks/complaint: {len(chunks_df)/len(df):.2f}")


🔪 Chunking 1,814 complaints...
📋 Available columns: ['Complaint ID', 'Date received', 'Product', 'Issue', 'Company', 'State', 'Consumer complaint narrative', 'Text_Length']
📝 Using narrative column: 'Complaint ID'
🏷️ Using product column: 'Product'

✅ Created 0 chunks from 1,814 complaints
📊 Avg chunks per complaint: 0.00

📊 Product distribution in chunks:


KeyError: 'product'

In [ ]:
embeddings, model = generate_all_embeddings(chunks_df)
print(f"\n✅ Embeddings complete: {embeddings.shape}")

In [ ]:
index, metadata = create_vectorstore(embeddings, chunks_df)
search("credit card fraud", model, index, metadata)
create_final_summary(df, chunks_df, index)
print("\n✅ TASK 2 COMPLETE! Check vector_store/ and reports/")